# Richter's Predictor

Initial code is a copy of the example found here: http://drivendata.co/blog/richters-predictor-benchmark/

We'll then use an XGBoost model to get a better estimate, and also will look at engineering some features using the geocode.

The intention is to try to find a way to use the fact that some areas (geo locations) will have suffered more damage than others.  There seem to be too many level 3 geolocations for a tree-based algorithm to deal with effectively, but if we can somehow uncover information about the geolocation and encode it in a way that is easier for the tree to deal with then it may improve our scores

In [1]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# for preprocessing the data
from sklearn.preprocessing import StandardScaler, LabelEncoder
from collections import defaultdict
from sklearn.model_selection import train_test_split

# the model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV


/Users/Andrew/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import xgboost as xgb

In [3]:
DATA_DIR = Path('.', 'data')

In [4]:
train_values = pd.read_csv(DATA_DIR / 'train_values.csv', index_col='building_id')
train_labels = pd.read_csv(DATA_DIR / 'train_labels.csv', index_col='building_id')


In [108]:
test_values  = pd.read_csv(DATA_DIR / 'test_values.csv', index_col='building_id')

#### For some feature engineering we want to use the damage grades, so we'll join them here

In [6]:
train = train_values.join(train_labels)

#### For others we also want to include test values (eg. when encoding categorical) 

In [7]:
test_and_train_values = pd.concat([train_values,test_values])

In [ ]:
train_labels.head()

In [ ]:
train_values.head()

In [ ]:
test_and_train_values.dtypes

## Data exploration

In [ ]:
(train_labels.damage_grade
             .value_counts()
             .sort_index()
             .plot.bar(title="Number of Buildings with Each Damage Grade"))

In [ ]:
selected_features = ['foundation_type', 
                     'area_percentage', 
                     'height_percentage',
                     'count_floors_pre_eq',
                     'land_surface_condition',
                     'has_superstructure_cement_mortar_stone']

train_values_subset = train_values[selected_features]

In [ ]:
sns.pairplot(train_values_subset.join(train_labels), 
             hue='damage_grade')

In [8]:
secondary_uses = [
'has_secondary_use',
'has_secondary_use_agriculture',
'has_secondary_use_hotel',
'has_secondary_use_rental',
'has_secondary_use_institution',
'has_secondary_use_school',
'has_secondary_use_industry',
'has_secondary_use_health_post',
'has_secondary_use_gov_office',
'has_secondary_use_use_police',
'has_secondary_use_other'
]

structure = [
'has_superstructure_adobe_mud',
'has_superstructure_mud_mortar_stone',
'has_superstructure_stone_flag',
'has_superstructure_cement_mortar_stone',
'has_superstructure_mud_mortar_brick',
'has_superstructure_cement_mortar_brick',
'has_superstructure_timber',
'has_superstructure_bamboo',
'has_superstructure_rc_non_engineered',
'has_superstructure_rc_engineered',
'has_superstructure_other'
]



In [ ]:
for use in secondary_uses:
    print(use, train[train[use]==1]['damage_grade'].mean())

In [ ]:
for s in structure:
    print(s, train[train[s]==1]['damage_grade'].mean())

In [ ]:
biggest_geo3 = train['geo_level_3_id'].value_counts().head(30).index.values

In [ ]:
for location in biggest_geo3:
    print('Geo3 id:',location)
    for s in structure:
        s_filter = (train['geo_level_3_id'] == location) & (train[s] == 1)
        print(s, train.loc[s_filter]['damage_grade'].count(), train.loc[s_filter]['damage_grade'].mean())

### There are similarities in damage between the mortar types (mud/cement) and the reinforced concrete types (non-eng, engineered) so for the sake of our geoid indicator we'll group them

In [109]:

train['mud'] = train['has_superstructure_adobe_mud'] | train['has_superstructure_mud_mortar_stone'] | train['has_superstructure_mud_mortar_brick']
train['cement'] = train['has_superstructure_cement_mortar_stone'] | train['has_superstructure_cement_mortar_brick'] 
train['concrete'] = train['has_superstructure_rc_non_engineered'] | train['has_superstructure_rc_engineered'] 
train['natural'] = train['has_superstructure_timber'] | train['has_superstructure_bamboo'] 

test_and_train_values['mud'] = test_and_train_values['has_superstructure_adobe_mud'] | test_and_train_values['has_superstructure_mud_mortar_stone'] | test_and_train_values['has_superstructure_mud_mortar_brick']
test_and_train_values['cement'] = test_and_train_values['has_superstructure_cement_mortar_stone'] | test_and_train_values['has_superstructure_cement_mortar_brick'] 
test_and_train_values['concrete'] = test_and_train_values['has_superstructure_rc_non_engineered'] | test_and_train_values['has_superstructure_rc_engineered'] 
test_and_train_values['natural'] = test_and_train_values['has_superstructure_timber'] | test_and_train_values['has_superstructure_bamboo'] 

test_values['mud'] = test_values['has_superstructure_adobe_mud'] | test_values['has_superstructure_mud_mortar_stone'] | test_values['has_superstructure_mud_mortar_brick']
test_values['cement'] = test_values['has_superstructure_cement_mortar_stone'] | test_values['has_superstructure_cement_mortar_brick'] 
test_values['concrete'] = test_values['has_superstructure_rc_non_engineered'] | test_values['has_superstructure_rc_engineered'] 
test_values['natural'] = test_values['has_superstructure_timber'] | test_values['has_superstructure_bamboo'] 




### Some geolocations might only be in the test set, so if we are going to build a universal lookup then we need to include test as well so we can get a complete list

In [ ]:
test_and_train_values.shape

In [ ]:
#test_and_train.head()
test_and_train_values.tail()

In [10]:
geo_lookup = test_and_train_values[['geo_level_3_id','geo_level_2_id','geo_level_1_id']].groupby(['geo_level_3_id']).first().reset_index()

In [ ]:
geo_lookup.head()

#### OK, now we can calcualte averages for each geoid level and construction type, and build our lookup table

First I'm going to double check what the frequency is like for the different structure types, because I'm worried that we won't have many examples of concrete and if that is an issue then we need to account for it somehow.

In [ ]:
# plot
f, axes = plt.subplots(1, 4, figsize=(14, 4), sharex=True, sharey=True)

# Change the x-axis because it has a really long tail.  First attempt was to make it log, second just trims
#axes[0,0].set(xscale="log")
axes[0].set_xlim(right=20)

graph_colours = ['skyblue','olive', 'gold', 'teal']
ax = [axes[0],axes[1],axes[2],axes[3]]

for i,s in enumerate(structure_cats):
    sns.distplot( geo_lookup[s+'1_n'].fillna(0) , color=graph_colours[i], ax=ax[i], bins=300)

#################################
f, axes = plt.subplots(1, 4, figsize=(14, 4), sharex=True, sharey=True)

# Change the x-axis because it has a really long tail.  First attempt was to make it log, second just trims
#axes[0,0].set(xscale="log")
axes[0].set_xlim(right=20)

graph_colours = ['skyblue','olive', 'gold', 'teal']
ax = [axes[0],axes[1],axes[2],axes[3]]

for i,s in enumerate(structure_cats):
    sns.distplot( geo_lookup[s+'2_n'].fillna(0) , color=graph_colours[i], ax=ax[i], bins=300)


#################################
f, axes = plt.subplots(2, 2, figsize=(7, 7), sharex=True, sharey=True)

# Change the x-axis because it has a really long tail.  First attempt was to make it log, second just trims
#axes[0,0].set(xscale="log")
axes[0,0].set_xlim(right=20)

graph_colours = ['skyblue','olive', 'gold', 'teal']
ax = [axes[0, 0],axes[0, 1],axes[1, 0],axes[1, 1]]


for i,s in enumerate(structure_cats):
    sns.distplot( geo_lookup[s+'3_n'].fillna(0) , color=graph_colours[i], ax=ax[i], bins=300)


In [ ]:
structure_cats = ['mud','natural', 'cement', 'concrete']

for location in biggest_geo3:
    print('Geo3 id:',location)
    for s in structure_cats:
        s_filter = (train['geo_level_3_id'] == location) & (train[s] == 1)
        print(s, train.loc[s_filter]['damage_grade'].count(), train.loc[s_filter]['damage_grade'].mean())

In [ ]:
averages = {}
levels = ['1','2','3']
for level in levels:
    for s in structure_cats:
        s_filter = train[s] == 1
        averages[s+level] = train[s_filter].groupby('geo_level_'+level+'_id')['damage_grade'].agg({s+level+'_n':'count', 
                                     s+level+'_mean':'mean'})

In [ ]:
averages['mud1'].head()

In [ ]:
train.merge(averages['mud2'].reset_index(), how='left',on='geo_level_2_id').head(2)

#### Everything is looking OK, let's do it again but this time merge inline rather than saving to a dictionary first
We use train here because that df has got the damage values in it

In [11]:
levels = ['1','2','3']
structure_cats = ['mud','natural', 'cement', 'concrete']


for level in levels:
    print('averaging level',level)
    averages_list = []

    # Work out normalised damage grades for each structure type
    for s in structure_cats:
        s_filter = train[s] == 1
        averages = train[s_filter].groupby('geo_level_'+level+'_id')['damage_grade'].agg({s+level+'_n':'count', 
                                     s+level+'_mean':'mean'})
        col_to_norm = averages[s+level+'_mean']
        averages[s+level+'_mean_norm']=(col_to_norm-col_to_norm.min())/(col_to_norm.max()-col_to_norm.min())
        #print(averages.head(2))
        averages_list.append(averages)

    # Concat the averages into one dataframe
    averages = pd.concat(averages_list, axis=1)
    #print(averages.head())
    #print(geo_lookup.shape)
    
    # Now we have those, we can also calculate a weighted avergage across the structure types for that geoid
    
    cols = [s+level+'_mean_norm' for s in structure_cats]
    weights = [s+level+'_n' for s in structure_cats]

    norms_np = averages[cols].values
    weights_np = averages[weights].values

    norm_mask = np.isnan(norms_np)
    weights_mask = np.isnan(weights_np)

    norms_np = np.ma.masked_array(norms_np, mask=norm_mask)
    weights_np = np.ma.masked_array(weights_np, mask=weights_mask)

    wa_norm = np.ma.average(norms_np, weights=weights_np, axis=1)
    wa_norm.fill_value = -1
    averages['level'+level+'norm_damage'] = wa_norm.filled()
    #geo_lookup['level'+level+'_wa_norm_damage'] = wa_norm.filled()
    #print(averages.head())
    geo_lookup = geo_lookup.merge(averages['level'+level+'norm_damage'].reset_index(), how='left',on='geo_level_'+level+'_id')

print('Done')

averaging level 1


/Users/Andrew/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  del sys.path[0]


averaging level 2
averaging level 3
Done


### Some  geoids have missing values though, presumably because the only examples are in the training set.  So we can use the next available level up

In [ ]:
geo_lookup.loc[8313]

In [12]:
empty_level = geo_lookup['level2norm_damage'].isnull()
geo_lookup.loc[empty_level,'level2norm_damage'] = geo_lookup.loc[empty_level,'level1norm_damage']

empty_level = geo_lookup['level3norm_damage'].isnull()
geo_lookup.loc[empty_level,'level3norm_damage'] = geo_lookup.loc[empty_level,'level2norm_damage']



In [ ]:
geo_lookup.loc[8313]

### Finally, join the lookup table with the test/train values

In [ ]:
test_and_train_values = test_and_train_values.merge(geo_lookup[['geo_level_3_id','level3norm_damage']], on=['geo_level_3_id'])

In [ ]:
test_and_train_values[22011:22015]

In [ ]:
test_and_train_values.index

In [ ]:
test_values.tail()

### Encode the categorical into numeric

This code taken from https://stackoverflow.com/questions/24458645/label-encoding-across-multiple-columns-in-scikit-learn

categorical_encoder = defaultdict(LabelEncoder)

'# Encoding the variable

fit = df.apply(lambda x: categorical_encoder[x.name].fit_transform(x))

'# Inverse the encoded

fit.apply(lambda x: categorical_encoder[x.name].inverse_transform(x))

'# Using the dictionary to label future data

df.apply(lambda x: categorical_encoder[x.name].transform(x))


In [13]:
categorical_columns = [
    'land_surface_condition',
    'foundation_type',
    'roof_type',
    'ground_floor_type',
    'other_floor_type',
    'position',
    'plan_configuration',
    'legal_ownership_status'
]

In [14]:
categorical_encoder = defaultdict(LabelEncoder)

### Fit the encoder on the combined dataset

In [15]:
test_and_train_values.loc[:,categorical_columns] = test_and_train_values.loc[:,categorical_columns].apply(lambda x: categorical_encoder[x.name].fit_transform(x))

In [ ]:
test_and_train_values.head()

### Apply the encoder to transform the train data

In [ ]:
train_values.loc[:,categorical_columns] = train_values.loc[:,categorical_columns].apply(lambda x: categorical_encoder[x.name].transform(x))

In [ ]:
train_values.dtypes

In [ ]:
train.head()

### All features are now categorical so we could use them on a classifier now, but...let's engineer some features first

In [16]:
train = train.merge(geo_lookup[['geo_level_3_id','level3norm_damage']], on=['geo_level_3_id'], how='left')

In [17]:
train.loc[:,categorical_columns] = train.loc[:,categorical_columns].apply(lambda x: categorical_encoder[x.name].transform(x))

In [ ]:
train.head()

### Prep the test data too

In [110]:
test_values = test_values.merge(geo_lookup[['geo_level_3_id','level3norm_damage']], on=['geo_level_3_id'], how='left')
test_values.loc[:,categorical_columns] = test_values.loc[:,categorical_columns].apply(lambda x: categorical_encoder[x.name].transform(x))

In [111]:
test_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,mud,cement,concrete,natural,level3norm_damage
0,17,596,11307,3,20,7,6,2,2,0,...,0,0,0,0,0,1,0,0,0,0.815789
1,6,141,11987,2,25,13,5,2,2,0,...,0,0,0,0,0,1,0,0,0,0.500000
2,22,19,10044,2,5,4,5,2,2,0,...,0,0,0,0,0,1,0,0,0,1.000000
3,26,39,633,1,0,19,3,2,2,2,...,0,0,0,0,0,0,1,0,0,0.222147
4,17,289,7970,3,15,8,7,2,2,1,...,0,0,0,0,0,1,0,0,0,0.916667


In [24]:
preds

array([1., 1., 2., ..., 0., 1., 2.], dtype=float32)

### Damage measures for each geolocation

The idae here is that I want a normalised measure of average damage per geolocation.  The easiest way would be to take the average damage value for the geoid but that wouldn't take into consideration the different mix of building types.  If some geolocations had sturdier buildings then it's average damage might be artifically low.

So instead I'll take the average damage for each building type and/or some sort of adjustment for the building type - for example wooden buildings seem to have less damage so perhaps we can work out some normalised values for each building type and then combine them to get a single normalised damage value for each geoid.

The last factor I want to allow for is that some geoids have only one data point, and that isn't gong to be of much use, so instead my intial approach will be to take the average of the next geolevel up if the count of datapoints is below a certain threshold

## Train Test Split

In [18]:
train_ex_geo = train.drop(['geo_level_1_id','geo_level_2_id','geo_level_3_id', 'damage_grade'], axis=1)

In [90]:
X_train, X_test, y_train, y_test = train_test_split(train_ex_geo, train['damage_grade'], test_size=0.2)

## Try random forest classifier

In [ ]:
clf = RandomForestClassifier(n_estimators=500, max_depth=10)
clf.fit(X_train, y_train) 

In [ ]:
print(clf.feature_importances_)

In [ ]:

y_pred = clf.predict(X_test)
f1_score(y_test, y_pred, average='micro')

## OK, now let's try that with XGBoost

In [91]:
# read in data
dtrain = xgb.DMatrix(X_train, label=y_train-1)
dtest = xgb.DMatrix(X_test, label=y_test-1)


In [96]:
# specify parameters via map
param = {'max_depth':10, 'eta':0.3, 'subsample':1, 'objective':'multi:softmax', 'num_class':3 }
num_round = 100


In [93]:
bst = xgb.train(param, dtrain, num_round)


In [94]:
# make prediction
y_pred = bst.predict(dtest) + 1

In [95]:
f1_score(y_test, y_pred, average='micro')

0.7523646898562959

In [97]:
X_train, X_test, y_train, y_test = train_test_split(train_ex_geo, train['damage_grade'], test_size=0)

In [98]:
dtrain = xgb.DMatrix(X_train, label=y_train-1)


In [99]:
bst = xgb.train(param, dtrain, num_round)


## Make a submission

In [112]:
test_ex_geo = test_values.drop(['geo_level_1_id','geo_level_2_id','geo_level_3_id'], axis=1)

In [113]:
dtest = xgb.DMatrix(test_ex_geo)


In [114]:
y_pred = bst.predict(dtest) + 1

In [115]:
submission_format = pd.read_csv(DATA_DIR / 'submission_format.csv', index_col='building_id')


In [116]:
my_submission = pd.DataFrame(data=y_pred,
                             columns=submission_format.columns,
                             index=submission_format.index)


In [117]:
my_submission.head()

,damage_grade
building_id,
300051,3.0
99355,2.0
890251,3.0
745817,1.0
421793,3.0


In [120]:
my_submission.damage_grade = my_submission.damage_grade.astype(int)

In [121]:
my_submission.head()

,damage_grade
building_id,
300051,3
99355,2
890251,3
745817,1
421793,3


In [122]:
my_submission.to_csv('submission.csv')

In [123]:
!head submission.csv

building_id,damage_grade
300051,3
99355,2
890251,3
745817,1
421793,3
871976,2
691228,2
896100,3
343471,2
